In [9]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import ipywidgets as widgets
from IPython.display import display
from math import log, sqrt, exp
from scipy.stats import norm
import warnings

# ------------------------------------------------------
# Root-Finding Methods & Plot Functions (Enhanced)
# ------------------------------------------------------

def bisection_method(f, tol=1e-6, max_iter=100):
    """
    Bisection Method with automatic interval search.
    """
    value_a, value_b = [], []
    for i in range(-10, 11):
        try:
            fi = f(i)
        except Exception:
            continue
        if fi < 0:
            value_a.append(i)
        elif fi > 0:
            value_b.append(i)
    if not value_a or not value_b:
        raise ValueError("Couldn't find a valid interval with a sign change.")
    a = max(value_a, key=lambda x: -abs(x))
    b = min(value_b, key=lambda x: abs(x))
    ini_a, ini_b = a, b
    iterations, values = [], []
    for i in range(max_iter):
        c = (a + b) / 2.0
        iterations.append(i)
        values.append(c)
        if abs(f(c)) < tol or (b - a) / 2 < tol:
            return c, iterations, values, ini_a, ini_b
        if f(a) * f(c) < 0:
            b = c
        else:
            a = c
    return c, iterations, values, ini_a, ini_b

def bisection_plot(f, ax):
    root, iterations, values, ini_a, ini_b = bisection_method(f)
    ax.plot(iterations, values, marker='o', linestyle='-', label=f'Root: {root:.6f}')
    ax.set_xlabel(f'Iteration (Initial: {ini_a}, {ini_b})')
    ax.set_ylabel('Approximate Root')
    ax.set_title('Bisection Convergence')
    ax.legend(loc="upper right")

def regula_falsi_method(f, max_iter=100, tol=1e-6):
    """
    Regula Falsi Method.
    """
    value_a, value_b = [], []
    for i in range(-5, 5):
        try:
            fi = f(i)
        except Exception:
            continue
        if fi < 0:
            value_a.append(i)
        elif fi > 0:
            value_b.append(i)
    if not value_a or not value_b:
        raise ValueError("Couldn't find a valid interval.")
    a = max(value_a)
    b = max(value_b)
    ini_a, ini_b = a, b
    iterations, values = [], []
    for i in range(max_iter):
        c = (a * f(b) - b * f(a)) / (f(b) - f(a))
        iterations.append(i)
        values.append(c)
        if abs(f(c)) < tol or abs(b - a) / 2 < tol:
            return c, iterations, values, ini_a, ini_b
        if f(a) * f(c) < 0:
            b = c
        else:
            a = c
    return c, iterations, values, ini_a, ini_b

def regula_falsi_plot(f, ax):
    root, iterations, values, ini_a, ini_b = regula_falsi_method(f)
    ax.plot(iterations, values, marker='o', linestyle='-', label=f'Root: {root:.6f}')
    ax.set_xlabel(f'Iteration (Initial: {ini_a}, {ini_b})')
    ax.set_ylabel('Approximate Root')
    ax.set_title('Regula Falsi Convergence')
    ax.legend(loc="upper right")

def secant_method(f, max_iter=100, tol=1e-6):
    """
    Secant Method.
    """
    iterations, values = [], []
    sample_points = np.unique(np.round(np.linspace(-10, 10, 20)).astype(int))
    function_values = np.abs(f(sample_points))
    best_indices = np.argsort(function_values)[:2]
    a, b = sample_points[best_indices]
    ini_a, ini_b = a, b
    for i in range(max_iter):
        if f(b) - f(a) == 0:
            raise ValueError("Division by zero encountered in Secant method.")
        c = b - f(b) * (b - a) / (f(b) - f(a))
        iterations.append(i)
        values.append(c)
        if abs(c - b) < tol or abs(f(b)) < tol:
            return c, iterations, values, ini_a, ini_b
        a, b = b, c
    return c, iterations, values, ini_a, ini_b

def secant_plot(f, ax):
    root, iterations, values, ini_a, ini_b = secant_method(f)
    ax.plot(iterations, values, marker='o', linestyle='-', label=f'Root: {root:.6f}')
    ax.set_xlabel(f'Iteration (Initial: {ini_a}, {ini_b})')
    ax.set_ylabel('Approximate Root')
    ax.set_title('Secant Convergence')
    ax.legend(loc="upper right")

def newton_raphson_method(f_expr, max_iter=100, tol=1e-6):
    """
    Newton-Raphson Method using sympy for differentiation.
    """
    x = sp.symbols('x')
    f_prime = sp.diff(f_expr, x)
    f = sp.lambdify(x, f_expr, "numpy")
    df = sp.lambdify(x, f_prime, "numpy")
    best_x0 = None
    best_f_value = float('inf')
    for _ in range(10):
        x0_candidate = np.random.randint(-5, 6)
        if x0_candidate == 0:
            continue
        f_value = abs(f(x0_candidate))
        if f_value < best_f_value:
            best_f_value = f_value
            best_x0 = x0_candidate
    if best_x0 is None:
        raise ValueError("Could not find a suitable initial guess for Newton-Raphson.")
    x0 = best_x0
    iterations, values = [], []
    for i in range(max_iter):
        f_x0 = f(x0)
        df_x0 = df(x0)
        if abs(df_x0) < 1e-10:
            raise ValueError("Derivative too close to zero, may not converge.")
        x_n = x0 - f_x0 / df_x0
        iterations.append(i)
        values.append(x_n)
        if abs(x_n - x0) < tol:
            return x_n, iterations, values, best_x0
        x0 = x_n
    return x0, iterations, values, best_x0

def newton_raphson_plot(f_expr, ax):
    root, iterations, values, best_x0 = newton_raphson_method(f_expr)
    ax.plot(iterations, values, marker='o', linestyle='-', label=f'Root: {root:.6f}')
    ax.set_xlabel(f'Iteration (Initial: {best_x0})')
    ax.set_ylabel('Approximate Root')
    ax.set_title('Newton-Raphson Convergence')
    ax.legend(loc="upper right")

def brent_method(f, tol=1e-6, max_iter=100):
    """
    Brent's Method (simplified version using inverse quadratic interpolation with bisection fallback).
    """
    value_a, value_b = [], []
    for i in range(-10, 11):
        try:
            fi = f(i)
        except Exception:
            continue
        if fi < 0:
            value_a.append(i)
        elif fi > 0:
            value_b.append(i)
    if not value_a or not value_b:
        raise ValueError("Couldn't find a valid interval.")
    a = max(value_a, key=lambda x: -abs(x))
    b = min(value_b, key=lambda x: abs(x))
    ini_a, ini_b = a, b
    c, d = a, a
    mflag = True
    iterations, values = [], []
    for i in range(max_iter):
        if f(a) != f(c) and f(b) != f(c):
            s = (a * f(b) * f(c)) / ((f(a) - f(b)) * (f(a) - f(c))) + \
                (b * f(a) * f(c)) / ((f(b) - f(a)) * (f(b) - f(c))) + \
                (c * f(a) * f(b)) / ((f(c) - f(a)) * (f(c) - f(b)))
        else:
            s = b - f(b) * (b - a) / (f(b) - f(a))
        cond1 = (s < (3 * a + b) / 4 or s > b)
        cond2 = mflag and abs(s - b) >= abs(b - c) / 2
        cond3 = not mflag and abs(s - b) >= abs(c - d) / 2
        cond4 = mflag and abs(b - c) < tol
        cond5 = not mflag and abs(c - d) < tol
        if cond1 or cond2 or cond3 or cond4 or cond5:
            s = (a + b) / 2
            mflag = True
        else:
            mflag = False
        iterations.append(i)
        values.append(s)
        fs = f(s)
        d, c = c, b
        if f(a) * fs < 0:
            b = s
        else:
            a = s
        if abs(f(a)) < abs(f(b)):
            a, b = b, a
        if abs(f(b)) < tol:
            return b, iterations, values, ini_a, ini_b
    return b, iterations, values, ini_a, ini_b

def brent_plot(f, ax):
    root, iterations, values, ini_a, ini_b = brent_method(f)
    ax.plot(iterations, values, marker='o', linestyle='-', label=f'Root: {root:.6f}')
    ax.set_xlabel(f'Iteration (Initial: {ini_a}, {ini_b})')
    ax.set_ylabel('Approximate Root')
    ax.set_title("Brent's Convergence")
    ax.legend(loc="upper right")

def halley_method(f_expr, tol=1e-6, max_iter=100):
    """
    Halley's Method using sympy for second derivative.
    """
    x = sp.symbols('x')
    f_prime = sp.diff(f_expr, x)
    f_double_prime = sp.diff(f_prime, x)
    f_numeric = sp.lambdify(x, f_expr, 'numpy')
    f_prime_numeric = sp.lambdify(x, f_prime, 'numpy')
    f_double_prime_numeric = sp.lambdify(x, f_double_prime, 'numpy')
    best_x0 = None
    best_f_value = float('inf')
    for i in range(-10, 11):
        try:
            f_val = abs(f_numeric(i))
        except Exception:
            continue
        if f_val < best_f_value:
            best_f_value = f_val
            best_x0 = i
    if best_x0 is None:
        raise ValueError("No suitable initial guess found for Halley's method.")
    x_n = best_x0
    iterations, values = [], []
    for i in range(max_iter):
        fx_n = f_numeric(x_n)
        fpx_n = f_prime_numeric(x_n)
        fppx_n = f_double_prime_numeric(x_n)
        iterations.append(i)
        values.append(x_n)
        if abs(fx_n) < tol:
            return x_n, iterations, values, best_x0
        denominator = 2 * fpx_n**2 - fx_n * fppx_n
        if abs(denominator) < 1e-12:
            raise ValueError("Halley's method failed: near zero denominator.")
        x_next = x_n - (2 * fx_n * fpx_n) / denominator
        if abs(x_next - x_n) < tol:
            return x_next, iterations, values, best_x0
        x_n = x_next
    return x_n, iterations, values, best_x0

def halley_plot(f_expr, ax):
    root, iterations, values, best_x0 = halley_method(f_expr)
    ax.plot(iterations, values, marker='o', linestyle='-', label=f'Root: {root:.6f}')
    ax.set_xlabel(f'Iteration (Initial: {best_x0})')
    ax.set_ylabel('Approximate Root')
    ax.set_title("Halley's Convergence")
    ax.legend(loc="upper right")

def muller(f, tol=1e-6, max_iter=100):
    """
    Müller's Method.
    """
    iterations, values = [], []
    sample_points = np.unique(np.round(np.linspace(-10, 10, 20)).astype(int))
    function_values = np.abs(f(sample_points))
    best_indices = np.argsort(function_values)[:3]
    x0, x1, x2 = sample_points[best_indices]
    ini_x0, ini_x1, ini_x2 = x0, x1, x2
    for i in range(max_iter):
        h1 = x1 - x0
        h2 = x2 - x1
        if h1 == 0 or h2 == 0:
            raise ValueError("Zero interval encountered in Müller's method.")
        del1 = (f(x1) - f(x0)) / h1
        del2 = (f(x2) - f(x1)) / h2
        a = (del2 - del1) / (h2 + h1)
        b = a * h2 + del2
        c = f(x2)
        disc = b**2 - 4 * a * c
        if disc >= 0:
            D = np.sqrt(disc)
        else:
            D = np.sqrt(-disc) * 1j
        E = b + D if abs(b + D) > abs(b - D) else b - D
        if E == 0:
            raise ValueError("Muller's method failed due to zero denominator.")
        x_new = x2 - (2 * c) / E
        iterations.append(i)
        values.append(x_new)
        if abs(x_new - x2) < tol:
            return x_new, iterations, values, ini_x0, ini_x1, ini_x2
        x0, x1, x2 = x1, x2, x_new
    return x2, iterations, values, ini_x0, ini_x1, ini_x2

def muller_plot(f, ax):
    root, iterations, values, ini_x0, ini_x1, ini_x2 = muller(f)
    ax.plot(iterations, values, marker='o', linestyle='-', label=f'Root: {root:.6f}')
    ax.set_xlabel(f'Iteration (Initial: {ini_x0}, {ini_x1}, {ini_x2})')
    ax.set_ylabel('Approximate Root')
    ax.set_title("Muller's Convergence")
    ax.legend(loc="upper right")

def bisection_method_interval(f, a, b, tol=1e-6, max_iter=100):
    """
    Bisection method applied to a specified interval.
    """
    iterations, values = [], []
    for i in range(max_iter):
        c = (a + b) / 2.0
        iterations.append(i)
        values.append(c)
        if abs(f(c)) < tol or (b - a) / 2 < tol:
            return c, iterations, values, a, b
        if f(a) * f(c) < 0:
            b = c
        else:
            a = c
    return c, iterations, values, a, b

def find_multiple_roots(f, x_min=-10, x_max=10, steps=100, tol=1e-6):
    """
    Find multiple roots in a specified interval.
    """
    roots = []
    x_vals = np.linspace(x_min, x_max, steps)
    for i in range(len(x_vals)-1):
        a = x_vals[i]
        b = x_vals[i+1]
        if f(a) * f(b) < 0:
            try:
                root, _, _, _, _ = bisection_method_interval(f, a, b, tol=tol)
                if not any(np.isclose(root, r, atol=tol) for r in roots):
                    roots.append(root)
            except Exception as e:
                warnings.warn(f"Error finding root in interval [{a}, {b}]: {e}")
    return roots

def multiple_roots_plot(f, ax, x_min=-10, x_max=10):
    xs = np.linspace(x_min, x_max, 400)
    ys = f(xs)
    ax.plot(xs, ys, label="f(x)")
    roots = find_multiple_roots(f, x_min, x_max)
    for r in roots:
        ax.plot(r, f(r), 'ro')
    ax.set_title("Multiple Roots (Red dots)")
    ax.set_xlabel("x")
    ax.set_ylabel("f(x)")
    ax.legend(loc="upper right")

# ------------------------------------------------------
# Implied Volatility Functions (Enhanced)
# ------------------------------------------------------

def black_scholes_call_price(S, K, r, T, sigma):
    """
    Black-Scholes call option price.
    """
    d1 = (log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*sqrt(T))
    d2 = d1 - sigma*sqrt(T)
    return S * norm.cdf(d1) - K * exp(-r*T) * norm.cdf(d2)

def implied_volatility(market_price, S, K, r, T, tol=1e-6, max_iter=100):
    """
    Compute implied volatility using Newton-Raphson, with convergence tracking.
    """
    sigma = 0.2  # initial guess
    sigmas = [sigma]
    for i in range(max_iter):
        price = black_scholes_call_price(S, K, r, T, sigma)
        d1 = (log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*sqrt(T))
        vega = S * sqrt(T) * norm.pdf(d1)
        if abs(vega) < 1e-8:
            raise ValueError("Vega is too low; Newton-Raphson may not converge.")
        diff = price - market_price
        if abs(diff) < tol:
            return sigma, sigmas, i
        sigma = sigma - diff / vega
        sigmas.append(sigma)
    return sigma, sigmas, max_iter

def implied_volatility_plot(market_price, S, K, r, T, ax):
    sigma, sigmas, iters = implied_volatility(market_price, S, K, r, T)
    ax.plot(range(len(sigmas)), sigmas, marker='o', linestyle='-',
            label=f'Implied Vol: {sigma:.4f}')
    ax.set_xlabel("Iteration")
    ax.set_ylabel("Sigma")
    ax.set_title("Implied Volatility Convergence")
    ax.legend(loc="upper right")

# ------------------------------------------------------
# Interactive Dashboard Setup
# ------------------------------------------------------

x = sp.symbols('x')

# --- Widgets for Root Finding ---
function_input = widgets.Text(
    value="x**3 - 2*x - 5", 
    description="f(x):",
    style={'description_width': 'initial'}
)
multiple_roots_checkbox = widgets.Checkbox(
    value=False,
    description="Find Multiple Roots in [-10,10]",
    indent=False
)
update_button = widgets.Button(description="Update Root Dashboard")

def process_function(change=None):
    user_function = function_input.value  
    try:
        function = sp.sympify(user_function)
        f_numeric = sp.lambdify(x, function, "numpy")
        return function, user_function, f_numeric
    except Exception as e:
        print("\nInvalid function input. Please enter a valid mathematical expression.")
        return None, None, None

def plot_root_dashboard(_):
    function, user_function, f_numeric = process_function()
    if f_numeric is None:
        return
    fig, axes = plt.subplots(3, 3, figsize=(15, 12))
    fig.suptitle(f"Root-Finding Methods for f(x) = {user_function}", fontsize=16)
    
    # Dictionary mapping method names to (plot_function, function argument, subplot axis)
    methods = {
        "Bisection": (bisection_plot, f_numeric, axes[0, 0]),
        "Regula Falsi": (regula_falsi_plot, f_numeric, axes[0, 1]),
        "Müller's": (muller_plot, f_numeric, axes[0, 2]),
        "Brent's": (brent_plot, f_numeric, axes[1, 0]),
        "Halley's": (halley_plot, function, axes[1, 1]),
        "Newton-Raphson": (newton_raphson_plot, function, axes[1, 2]),
        "Secant": (secant_plot, f_numeric, axes[2, 0])
    }
    
    for method, (plot_func, func, ax) in methods.items():
        try:
            plot_func(func, ax)
            ax.grid(True)
        except Exception as e:
            ax.text(0.5, 0.5, f"Error in {method}:\n{e}", ha='center')
            print(f"Error in {method}: {e}")
    
    # If user wants to find multiple roots, show an extra plot.
    if multiple_roots_checkbox.value:
        ax_extra = axes[2, 1]
        try:
            multiple_roots_plot(f_numeric, ax_extra)
        except Exception as e:
            ax_extra.text(0.5, 0.5, f"Error:\n{e}", ha='center')
            print(f"Error in Multiple Roots: {e}")
    
    # Remove any unused subplots.
    for i in range(len(methods) + (1 if multiple_roots_checkbox.value else 0), 9):
        fig.delaxes(axes.flat[i])
    
    plt.tight_layout()
    plt.show()

update_button.on_click(plot_root_dashboard)
root_tab = widgets.VBox([function_input, multiple_roots_checkbox, update_button])

# --- Widgets for Implied Volatility ---
S_input = widgets.FloatText(value=100.0, description="Underlying Price (S):", style={'description_width': 'initial'})
K_input = widgets.FloatText(value=100.0, description="Strike Price (K):", style={'description_width': 'initial'})
r_input = widgets.FloatText(value=0.05, description="Risk-Free Rate (r):", style={'description_width': 'initial'})
T_input = widgets.FloatText(value=1.0, description="Time to Maturity (T, years):", style={'description_width': 'initial'})
market_price_input = widgets.FloatText(value=10.0, description="Market Call Price:", style={'description_width': 'initial'})
iv_update_button = widgets.Button(description="Compute Implied Volatility")

def plot_iv_dashboard(_):
    S = S_input.value
    K = K_input.value
    r = r_input.value
    T = T_input.value
    market_price = market_price_input.value
    
    fig, ax = plt.subplots(figsize=(7, 5))
    try:
        implied_volatility_plot(market_price, S, K, r, T, ax)
        ax.grid(True)
        fig.suptitle("Implied Volatility via Newton-Raphson", fontsize=14)
        plt.show()
    except Exception as e:
        print(f"Error computing implied volatility: {e}")

iv_update_button.on_click(plot_iv_dashboard)
iv_tab = widgets.VBox([S_input, K_input, r_input, T_input, market_price_input, iv_update_button])

# --- Combine Tabs in a Tab Widget ---
tab = widgets.Tab(children=[root_tab, iv_tab])
tab.set_title(0, "Root Finding")
tab.set_title(1, "Implied Volatility")
display(tab)